In [1]:
import torch
from torch import nn, optim
import os 
import torch.nn.functional as F

In [2]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
encoder.load_model(encoder_weights)
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

D:\pycharm_project\vs2ttsvoice\encoder\audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Loaded encoder "pretrained.pt" trained to step 1564501
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder\saved_models\pretrained\pretrained.pt


In [3]:
def embedEX(in_fpath):
    filepath=r'D:\voiceset\dataset\test_wav'
#     filepath=r'D:\voiceset\dataset\wav'
    in_fpath=os.path.join(filepath,in_fpath[:7],in_fpath[8:-6],in_fpath[-5:]+'.wav')
    reprocessed_wav = encoder.preprocess_wav(in_fpath)
    original_wav, sampling_rate = librosa.load(in_fpath)
    preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
    embed,_ = encoder.embed_utterance(preprocessed_wav)
    return embed

In [4]:
class X2E_NetL1L(torch.nn.Module):
    def __init__(self, in_dim, n_hidden_1,n_hidden_2,n_hidden_3,n_hidden_4,out_dim):
        super(X2E_NetL1L, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Linear(in_dim, n_hidden_1), torch.nn.ReLU(True))
        self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, n_hidden_2), torch.nn.ReLU(True))  
        self.layer3 = torch.nn.Sequential(torch.nn.Linear(n_hidden_2, n_hidden_3), torch.nn.ReLU(True))
        self.layer4 = torch.nn.Sequential(torch.nn.Linear(n_hidden_3, n_hidden_4), torch.nn.ReLU(True))
        self.layer5 = torch.nn.Sequential(torch.nn.Linear(n_hidden_4, out_dim))
 
    def forward(self, x):
        x=x/torch.norm(x, p=2)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x=x/torch.norm(x, p=2)
        return x

In [5]:
#模型
model=X2E_NetL1L(512,450,400,350,300,256)
#损失函数
# criterion = torch.nn.MSELoss(reduce=True, size_average=False)
# criterion = torch.nn.MSELoss(reduction='sum')
criterion = torch.nn.L1Loss(reduction='sum')
#优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
# 训练模型
epoch = 0
totaldir='D:\\pycharm_project\\vs2ttsvoice\\X2E'
#损失值写入文件

for i in range(80):
    pathstr ='xvector'+str(i+1)+'.txt' 
    pathstr=os.path.join(totaldir, pathstr)
    file=open( pathstr)
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        filename=data[0]
        data=data[1:]
        out=list(map(float,data))
        out=torch.tensor(out)
        pred=model(out)
        label=embedEX(filename)
        label=torch.from_numpy(label)
        loss = criterion(pred, label)
        print_loss = loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch+=1
        if epoch%100 == 0:
        #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
        #保存参数、
            print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
            state = { 'model':model.state_dict(), 'epoch':epoch }                       
            if not os.path.isdir('checkpointX2E'):
                 os.mkdir('checkpointX2E')    
            torch.save(state, '.\checkpointX2E\model2L1L_epoch_%d.ckpt' % (epoch))
            file_save = open('lossX2EL1L.txt',mode='a')
            file_save.write('\n'+'epoch:'+str(epoch)+'  loss:'+str(loss.data.item()))
            file_save.close() 

epoch: 100, loss: 6.028
epoch: 200, loss: 5.096
epoch: 300, loss: 3.261
epoch: 400, loss: 7.451
epoch: 500, loss: 6.615
epoch: 600, loss: 4.224
epoch: 700, loss: 4.547
epoch: 800, loss: 5.113
epoch: 900, loss: 5.043
epoch: 1000, loss: 5.851
epoch: 1100, loss: 7.461
epoch: 1200, loss: 6.291
epoch: 1300, loss: 6.404
epoch: 1400, loss: 6.805
epoch: 1500, loss: 5.77
epoch: 1600, loss: 3.57
epoch: 1700, loss: 8.079
epoch: 1800, loss: 5.313
epoch: 1900, loss: 4.797
epoch: 2000, loss: 5.643
epoch: 2100, loss: 3.938
epoch: 2200, loss: 7.819
epoch: 2300, loss: 3.358
epoch: 2400, loss: 5.788
epoch: 2500, loss: 3.756
epoch: 2600, loss: 5.209
epoch: 2700, loss: 3.096
epoch: 2800, loss: 8.514
epoch: 2900, loss: 3.012
epoch: 3000, loss: 5.977
epoch: 3100, loss: 3.784
epoch: 3200, loss: 5.528
epoch: 3300, loss: 8.224
epoch: 3400, loss: 5.322
epoch: 3500, loss: 6.8
epoch: 3600, loss: 4.355
epoch: 3700, loss: 5.961
epoch: 3800, loss: 8.211
epoch: 3900, loss: 6.103
epoch: 4000, loss: 7.065
epoch: 4100, 

epoch: 32200, loss: 6.002
epoch: 32300, loss: 5.708
epoch: 32400, loss: 5.186
epoch: 32500, loss: 6.814
epoch: 32600, loss: 5.193
epoch: 32700, loss: 6.225
epoch: 32800, loss: 4.223
epoch: 32900, loss: 4.737
epoch: 33000, loss: 6.28
epoch: 33100, loss: 4.585
epoch: 33200, loss: 6.401
epoch: 33300, loss: 5.609
epoch: 33400, loss: 5.885
epoch: 33500, loss: 5.175
epoch: 33600, loss: 6.27
epoch: 33700, loss: 5.956
epoch: 33800, loss: 5.347
epoch: 33900, loss: 5.975
epoch: 34000, loss: 4.45
epoch: 34100, loss: 3.831
epoch: 34200, loss: 4.931
epoch: 34300, loss: 6.303
epoch: 34400, loss: 6.743
epoch: 34500, loss: 6.009
epoch: 34600, loss: 4.424
epoch: 34700, loss: 6.496
epoch: 34800, loss: 6.449
epoch: 34900, loss: 5.186
epoch: 35000, loss: 6.228
epoch: 35100, loss: 7.16
epoch: 35200, loss: 6.136
epoch: 35300, loss: 4.551
epoch: 35400, loss: 5.37
epoch: 35500, loss: 6.008
epoch: 35600, loss: 5.145
epoch: 35700, loss: 5.773
epoch: 35800, loss: 5.321
epoch: 35900, loss: 4.063
epoch: 36000, los

epoch: 63900, loss: 3.983
epoch: 64000, loss: 6.601
epoch: 64100, loss: 4.588
epoch: 64200, loss: 6.014
epoch: 64300, loss: 5.844
epoch: 64400, loss: 4.344
epoch: 64500, loss: 4.054
epoch: 64600, loss: 5.817
epoch: 64700, loss: 4.13
epoch: 64800, loss: 4.825
epoch: 64900, loss: 6.233
epoch: 65000, loss: 6.034
epoch: 65100, loss: 4.988
epoch: 65200, loss: 6.464
epoch: 65300, loss: 4.187
epoch: 65400, loss: 7.01
epoch: 65500, loss: 5.551
epoch: 65600, loss: 6.242
epoch: 65700, loss: 4.628
epoch: 65800, loss: 4.675
epoch: 65900, loss: 3.042
epoch: 66000, loss: 6.759
epoch: 66100, loss: 5.288
epoch: 66200, loss: 6.532
epoch: 66300, loss: 6.225
epoch: 66400, loss: 5.873
epoch: 66500, loss: 5.075
epoch: 66600, loss: 6.763
epoch: 66700, loss: 5.193
epoch: 66800, loss: 4.874
epoch: 66900, loss: 6.355
epoch: 67000, loss: 6.596
epoch: 67100, loss: 6.05
epoch: 67200, loss: 6.043
epoch: 67300, loss: 5.501
epoch: 67400, loss: 5.839
epoch: 67500, loss: 5.201
epoch: 67600, loss: 5.255
epoch: 67700, l

epoch: 95600, loss: 4.675
epoch: 95700, loss: 5.2
epoch: 95800, loss: 4.46
epoch: 95900, loss: 5.275
epoch: 96000, loss: 4.025
epoch: 96100, loss: 5.598
epoch: 96200, loss: 5.994
epoch: 96300, loss: 5.12
epoch: 96400, loss: 4.465
epoch: 96500, loss: 5.55
epoch: 96600, loss: 4.595
epoch: 96700, loss: 6.346
epoch: 96800, loss: 5.748
epoch: 96900, loss: 4.496
epoch: 97000, loss: 4.786
epoch: 97100, loss: 5.519
epoch: 97200, loss: 4.014
epoch: 97300, loss: 5.436
epoch: 97400, loss: 5.145
epoch: 97500, loss: 4.285
epoch: 97600, loss: 5.243
epoch: 97700, loss: 6.455
epoch: 97800, loss: 5.978
epoch: 97900, loss: 5.694
epoch: 98000, loss: 6.886
epoch: 98100, loss: 4.332
epoch: 98200, loss: 6.114
epoch: 98300, loss: 5.384
epoch: 98400, loss: 5.06
epoch: 98500, loss: 4.519
epoch: 98600, loss: 5.481
epoch: 98700, loss: 5.833
epoch: 98800, loss: 5.361
epoch: 98900, loss: 5.371
epoch: 99000, loss: 4.827
epoch: 99100, loss: 7.059
epoch: 99200, loss: 6.196
epoch: 99300, loss: 5.171
epoch: 99400, loss

epoch: 126300, loss: 3.928
epoch: 126400, loss: 4.711
epoch: 126500, loss: 6.629
epoch: 126600, loss: 3.997
epoch: 126700, loss: 6.272
epoch: 126800, loss: 4.671
epoch: 126900, loss: 4.302
epoch: 127000, loss: 6.43
epoch: 127100, loss: 6.633
epoch: 127200, loss: 4.842
epoch: 127300, loss: 4.503
epoch: 127400, loss: 4.768
epoch: 127500, loss: 6.137
epoch: 127600, loss: 3.857
epoch: 127700, loss: 5.142
epoch: 127800, loss: 5.161
epoch: 127900, loss: 5.713
epoch: 128000, loss: 5.966
epoch: 128100, loss: 6.079
epoch: 128200, loss: 7.479
epoch: 128300, loss: 5.56
epoch: 128400, loss: 6.731
epoch: 128500, loss: 6.058
epoch: 128600, loss: 5.609
epoch: 128700, loss: 5.107
epoch: 128800, loss: 4.763
epoch: 128900, loss: 4.669
epoch: 129000, loss: 5.012
epoch: 129100, loss: 4.699
epoch: 129200, loss: 6.542
epoch: 129300, loss: 5.413
epoch: 129400, loss: 4.733
epoch: 129500, loss: 6.391
epoch: 129600, loss: 6.107
epoch: 129700, loss: 5.328
epoch: 129800, loss: 4.35
epoch: 129900, loss: 4.89
epoch

In [12]:
# 读取训练参数
# checkpoint = torch.load(r'C:\NLP\tts_rtvc\rtvc\checkpoint\model3_epoch_56300.ckpt')
checkpoint = torch.load(r'D:\pycharm_project\vs2ttsvoice\checkpointX2E\model2L1L_epoch_98800.ckpt')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [13]:
#合成语音
from IPython.utils import io
epoch=0
text="This is being said in my own voice.The computer has learned to do an impression of me."
file=open('.\X2E\pv_xvector.txt')
filenamelabel=''
for line in file.readlines():
# line=file.readline()
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    if (filenamelabel==filename[:7]):
        continue
    print(filename)
    data=data[1:]
#     out=[[list(map(float,data))]]
#     out=torch.tensor(out)
#     out=out.permute(0,2,1)
#     pred=model(out)
#     pred=pred.squeeze(0)
#     pred=pred.tolist()
#     print("pred",pred)
    out=list(map(float,data))
    out=torch.tensor(out)
    pred=model(out)
    pred=pred.tolist()
#     print("pred",pred)
    print("Synthesizing 预测语音...")
    with io.capture_output() as captured:
      specs = synthesizer.synthesize_spectrograms([text], [pred])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
#     sf.write(filename[:7]+"A.wav", generated_wav, synthesizer.sample_rate)
    epoch =+1
    #原始声音合成
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    label=embedEX(filename)
    print("Synthesizing 原始语音...")
    print('阈值：',np.linalg.norm(pred-label))
    with io.capture_output() as captured:
       specs = synthesizer.synthesize_spectrograms([text], [label])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    filenamelabel=filename[:7]
#     sf.write(filename[:7]+".wav", generated_wav, synthesizer.sample_rate)

id10270-5r0dWxy17C8-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6543541540126337


id10271-1gtz-CUIygI-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.653715339823984


id10272-0qqQJVFKpbA-00001
Synthesizing 预测语音...


...

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6886318259920168


id10274-7rkVfW2yw0c-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6376235598868851


id10275-CVUXDNZzcmA-00001
Synthesizing 预测语音...


KeyboardInterrupt: 